In [1]:
# ================================================================
# BOOTSTRAP CELL FOR Integration_Pipeline.ipynb
# Loads NLP artifacts, classifier model, and dependencies
# ================================================================

!pip install langdetect pandas numpy scikit-learn joblib --quiet

import os
import json
import re
import pandas as pd
import numpy as np
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0
import joblib

# mount Drive
from google.colab import drive
drive.mount('/content/drive')

# Load NLP artifacts
artifact_path = "/content/drive/MyDrive/Colab Notebooks/Model Notebooks/artifacts/"
with open(artifact_path + "symptom_map.json", "r", encoding="utf-8") as f:
    symptom_map = json.load(f)
with open(artifact_path + "keyword_lookup.json", "r", encoding="utf-8") as f:
    keyword_lookup = json.load(f)

# Load ML model
model_path = "/content/drive/MyDrive/Colab Notebooks/Model Notebooks/model_artifacts/child_disease_random_forest.pkl"
rf_model = joblib.load(model_path)

print("Loaded classifier + NLP artifacts")

print("✅ Integration Notebook Bootstrap Loaded Successfully")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Mounted at /content/drive
Loaded classifier + NLP artifacts
✅ Integration Notebook Bootstrap Loaded Successfully


# **Integration Pipeline: NLP + Random Forest Classifier**
### Notebook: Integration_Pipeline.ipynb
### Scenario: VHT Childhood Disease Assistant

This notebook integrates:
1. The NLP Symptom Extraction Engine
2. The Random Forest Disease Classifier
3. Metadata analysis (age, duration)

Purpose:
- Convert free-text caregiver/VHT descriptions into structured symptoms.
- Use ML classifier to predict disease.
- Combine outputs into a single cognitive decision.

This creates the heart of the system that will power the final VHT assistant.


## **Mount Drive and Load Saved Artifacts**

We load:
- NLP symptom_map.json
- NLP keyword_lookup.json
- Trained Random Forest model (.pkl)


In [9]:
import os

for root, dirs, files in os.walk("/content/drive/MyDrive"):
    for f in files:
        if "child_disease_random_forest.pkl" in f:
            print("FOUND MODEL HERE →", os.path.join(root, f))


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [22]:
import json
import joblib
import os

nlp_artifacts_path = "/content/drive/MyDrive/Colab Notebooks/Model Notebooks/artifacts/"
model_path = "/content/drive/MyDrive/Colab Notebooks/Model Notebooks/model_artifacts/child_disease_random_forest.pkl"



with open(nlp_artifacts_path + "symptom_map.json", "r", encoding="utf-8") as f:
    symptom_map = json.load(f)

with open(nlp_artifacts_path + "keyword_lookup.json", "r", encoding="utf-8") as f:
    keyword_lookup = json.load(f)

rf_model = joblib.load(model_path)

print("Artifacts loaded successfully!")


Artifacts loaded successfully!


## **Rebuild the NLP Understanding Engine**

The integration pipeline requires the same NLP processing logic used in the
NLP_Symptom_Extractor.ipynb notebook.

We re-create:
- `clean_text()`
- `is_negated()`
- `extract_symptoms()`
- `nlp_understanding_pipeline()`

This allows the system to take raw Luganda/English text and convert it
into structured symptom data for the classifier.


In [14]:
!pip install langdetect


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=f7f6c7e49d501840c78068348d8f1c43a621c1a3ca1c9c5c70442c2236925524
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [24]:
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0


In [23]:
import re
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0  # ensures consistent language detection

negation_words = {"no", "not", "without", "tet", "si", "siri", "sita", "siko"}

def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9\u0100-\uffff\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def is_negated(tokens, idx, window=3):
    start = max(0, idx - window)
    end = min(len(tokens), idx + window + 1)
    return any(tokens[i] in negation_words for i in range(start, end))

def extract_symptoms(text):
    cleaned = clean_text(text)
    tokens = cleaned.split()

    detected = {symptom: False for symptom in symptom_map}

    for kw, symptom in keyword_lookup.items():
        if kw in cleaned:
            idx = cleaned.find(kw)
            token_index = len(cleaned[:idx].split())
            if not is_negated(tokens, token_index):
                detected[symptom] = True

    age_match = re.search(r"(\d{1,2})\s*year", cleaned)
    duration_match = re.search(r"(\d{1,2})\s*(day|days|d)", cleaned)

    metadata = {
        "age_years": int(age_match.group(1)) if age_match else None,
        "duration_days": int(duration_match.group(1)) if duration_match else None
    }

    return {"symptoms": detected, "metadata": metadata, "cleaned_text": cleaned}

def nlp_understanding_pipeline(text):
    try:
        lang = detect(text)
    except:
        lang = "unknown"

    result = extract_symptoms(text)
    result["language"] = lang
    return result


## **Convert NLP Output into Classifier Feature Vector**
The classifier expects numerical features.  
This function turns symptom booleans + metadata into a numeric vector.


In [25]:
def symptoms_to_features(nlp_output):
    symptoms = nlp_output["symptoms"]
    metadata = nlp_output["metadata"]

    feature_vector = [
        int(symptoms.get("fever", False)),
        int(symptoms.get("cough", False)),
        int(symptoms.get("fast_breathing", False)),
        int(symptoms.get("diarrhea", False)),
        int(symptoms.get("vomiting", False)),
        int(symptoms.get("weakness", False)),
        int(symptoms.get("poor_feeding", False)),
        int(symptoms.get("convulsions", False)),
        metadata["age_years"] if metadata["age_years"] else 2,
        metadata["duration_days"] if metadata["duration_days"] else 1
    ]

    return feature_vector


## **Full Pipeline Test: NLP + Classifier**
This test shows how free text becomes:
- Clean text
- Extracted symptoms
- Feature vector
- Disease prediction


In [28]:
test_text = "Omwana alina omusujja, akyawa okukosora era afuuya mangu."

nlp_out = nlp_understanding_pipeline(test_text)
features = symptoms_to_features(nlp_out)

prediction = rf_model.predict([features])[0]
probabilities = rf_model.predict_proba([features])[0]

print("Input Text:", test_text)
print("\nExtracted Symptoms:", nlp_out["symptoms"])
print("Metadata:", nlp_out["metadata"])
print("\nPredicted Disease:", prediction)
print("\nConfidence Scores:", probabilities)


Input Text: Omwana alina omusujja, akyawa okukosora era afuuya mangu.

Extracted Symptoms: {'fever': True, 'cough': True, 'fast_breathing': True, 'difficulty_breathing': False, 'diarrhea': False, 'vomiting': False, 'blood_in_stool': False, 'weakness': False, 'poor_feeding': False, 'convulsions': False}
Metadata: {'age_years': None, 'duration_days': None}

Predicted Disease: pneumonia

Confidence Scores: [0.    0.005 0.995]


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


## **Explanation Layer for Classifier Predictions**

This layer generates a human-readable explanation for the predicted disease.

It combines:
- Symptoms detected by the NLP engine
- Medical reasoning based on IMCI-style logic
- Classifier probability scores

Purpose:
- Increase transparency and interpretability
- Help VHTs understand *why* the system made its decision
- Support ethical AI by avoiding black-box predictions


In [29]:
def generate_explanation(nlp_output, prediction, probabilities):
    symptoms = nlp_output["symptoms"]

    explanation = []

    # Add classifier confidence
    explanation.append(f"Classifier confidence: {max(probabilities)*100:.1f}%")

    # Pneumonia rules
    if prediction == "pneumonia":
        if symptoms["fast_breathing"]:
            explanation.append("- Fast breathing detected → key pneumonia indicator (IMCI).")
        if symptoms["cough"]:
            explanation.append("- Cough present → supports pneumonia diagnosis.")
        if symptoms["fever"]:
            explanation.append("- Fever detected → common pneumonia co-symptom.")

    # Malaria rules
    elif prediction == "malaria":
        if symptoms["fever"]:
            explanation.append("- Fever detected → primary malaria symptom.")
        if symptoms["vomiting"]:
            explanation.append("- Vomiting present → common in malaria cases.")
        if symptoms["weakness"]:
            explanation.append("- Weakness/fatigue → associated with malaria.")

    # Diarrhea rules
    elif prediction == "diarrhea":
        if symptoms["diarrhea"]:
            explanation.append("- Diarrhea detected → defining symptom.")
        if symptoms["vomiting"]:
            explanation.append("- Vomiting → common in dehydration.")
        if symptoms["fever"]:
            explanation.append("- Fever present → possible infection.")

    # Default fallback
    if len(explanation) == 1:
        explanation.append("No strong rule-based signals detected; relying mainly on classifier confidence.")

    return explanation


In [30]:
test_text = "Omwana alina omusujja, akyawa okukosora era afuuya mangu."

nlp_out = nlp_understanding_pipeline(test_text)
features = symptoms_to_features(nlp_out)

prediction = rf_model.predict([features])[0]
probabilities = rf_model.predict_proba([features])[0]

explanation = generate_explanation(nlp_out, prediction, probabilities)

print("🩺 INPUT TEXT:", test_text)
print("\n📌 Predicted Disease:", prediction)
print("\n📋 Explanation:")
for line in explanation:
    print("  ", line)


🩺 INPUT TEXT: Omwana alina omusujja, akyawa okukosora era afuuya mangu.

📌 Predicted Disease: pneumonia

📋 Explanation:
   Classifier confidence: 99.5%
   - Fast breathing detected → key pneumonia indicator (IMCI).
   - Cough present → supports pneumonia diagnosis.
   - Fever detected → common pneumonia co-symptom.


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
